# Algorithm

First allocation algorithm implemented 🚀 (Only problem: I have no clue how it works 😂) 
- only events with one hall
- only one parking lot per event (actually not 100% sure)
- only one parking lot per event (thats why fre.e + IMOT doesn't have a parking lot)
- algorithm minimizes the distance in the entire dataframe not per day (must be fixed for sure)

In [1]:
# Import Libraries
import pulp as pl
import pandas as pd

In [2]:
# Load the data
df_events_closest_parking_lot_min_capacity = pd.read_parquet('../data/output_data/events_closest_parking_lot.parquet')

In [3]:
import pulp as pl

# Define the problem
model = pl.LpProblem("Minimize_Distance", pl.LpMinimize)

# Define decision variables
assignments = pl.LpVariable.dicts("Assign",
                                  ((event, date, parking_lot) for index, (event, date, parking_lot) in df_events_closest_parking_lot_min_capacity[
                                      ['event', 'date', 'parking_lot']].drop_duplicates().iterrows()),
                                  cat='Binary')

# Objective: Minimize the total minimum distance
model += pl.lpSum(assignments[(event, date, parking_lot)] * min_distance
                  for index, (event, date, parking_lot, min_distance) in df_events_closest_parking_lot_min_capacity[
                      ['event', 'date', 'parking_lot', 'min_distance']].iterrows())

# Constraint: Each event on each date should have exactly one parking lot assigned
for (event, date), group in df_events_closest_parking_lot_min_capacity.groupby(['event', 'date']):
    model += pl.lpSum(assignments[(event, date, parking_lot)] for parking_lot in group['parking_lot']) == 1

# Constraint: Do not exceed the capacity of any parking lot on any given day
for (parking_lot, date), group in df_events_closest_parking_lot_min_capacity.groupby(['parking_lot', 'date']):
    model += pl.lpSum(assignments[(event, date, parking_lot)] * demand
                      for index, (event, demand) in group[['event', 'demand']].iterrows()) <= group['capacity'].iloc[0]

# Solve the model
model.solve()

# Output results
results = []
for (event, date, parking_lot), var in assignments.items():
    if pl.value(var) == 1:
        results.append({"event": event, "date": date, "parking_lot": parking_lot})

results_df = pd.DataFrame(results)

# merge capacity onto results_df
results_df = results_df.merge(df_events_closest_parking_lot_min_capacity[['parking_lot']].drop_duplicates(), on='parking_lot')

results_df.head()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jakobschwarz/Documents/01_study/02_master/01_coursework/02_semester/02_Management and Digital Technologies II - Software Development Project (12)/development/southpark/venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0x/vxtbhltd3k1g7dgrlry82kfh0000gn/T/5ff39495564147199aad9bd044bd0eb8-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0x/vxtbhltd3k1g7dgrlry82kfh0000gn/T/5ff39495564147199aad9bd044bd0eb8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 134 COLUMNS
At line 1050 RHS
At line 1180 BOUNDS
At line 1366 ENDATA
Problem MODEL has 129 rows, 185 columns and 360 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18630.7 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 20 strengthened rows, 1 substitutions
Cgl0003I 0 fixed, 0 

,event,date,parking_lot
0,inhorgenta,2025-02-17,P1 Nord (westl. Tor 17a)
1,inhorgenta,2025-02-18,P1 Nord (westl. Tor 17a)
2,inhorgenta,2025-02-19,P1 Nord (Tor 17a - Tor 11c)
3,inhorgenta,2025-02-20,P1 Nord (Tor 17a - Tor 11c)
4,inhorgenta,2025-02-21,P1 Nord (Tor 17a - Tor 11c)


In [4]:
# Load events data for merging with the results
df_events = pd.read_csv('../data/input_data/events.csv')
df_events['date'] = pd.to_datetime(df_events['date'], format='%d.%m.%y')

In [5]:
# Merge the results with the events data
df_events_allocated_parking_lot = pd.merge(df_events, results_df, on=['event', 'date'], how='left')

In [6]:
# load df_halls_parking_distances
df_halls_parking_distances = pd.read_parquet('../data/output_data/halls_parking_distances.parquet')

In [7]:
# Merge df_events_allocated_parking_lot with df_events_closest_parking_lot_min_capacity to get distances
df_events_allocated_parking_lot = pd.merge(df_events_allocated_parking_lot, df_halls_parking_distances, on=['hall', 'parking_lot'], how='left')

In [8]:
# load parking lots capacity
df_parking_lots_capacity = pd.read_csv('../data/input_data/parking_lots_capacity.csv')

In [9]:
# Merge capacity from df_events_closest_parking_lot_min_capacity onto df_events_allocated_parking_lot

df_events_allocated_parking_lot = pd.merge(
    df_events_allocated_parking_lot, 
    df_events_closest_parking_lot_min_capacity[['parking_lot', 'date', 'capacity']], 
    on=['parking_lot', 'date'], 
    how='left'
).drop_duplicates()

In [10]:
# Add parking_delta column
df_events_allocated_parking_lot['parking_delta'] = df_events_allocated_parking_lot['capacity'] - df_events_allocated_parking_lot['demand']

# Add a column for the minimum distance and the entrance
df_events_allocated_parking_lot['min_distance'] = df_events_allocated_parking_lot[['distance_north', 'distance_north_east', 'distance_east', 'distance_west', 'distance_north_west']].min(axis=1)

# Check if there are any rows with all-NA values
mask = df_events_allocated_parking_lot[['distance_north', 'distance_north_east', 'distance_east', 'distance_west', 'distance_north_west']].notna().any(axis=1)

# Only calculate 'closest_entrance' for rows without all-NA values
df_events_allocated_parking_lot.loc[mask, 'closest_entrance'] = df_events_allocated_parking_lot.loc[mask, ['distance_north', 'distance_north_east', 'distance_east', 'distance_west', 'distance_north_west']].idxmin(axis=1).str.replace('distance_', '')

In [11]:
# First allocation algorithm results
df_events_allocated_parking_lot[df_events_allocated_parking_lot['date'] == pd.to_datetime('2025-02-23')][['event', 'date', 'status', 'hall', 'demand', 'parking_delta', 'capacity', 'parking_lot', 'min_distance', 'closest_entrance', 'distance_north', 'distance_north', 'distance_north_east', 'distance_east', 'distance_west', 'distance_north_west', ]].head(20)

,event,date,status,hall,demand,parking_delta,capacity,parking_lot,min_distance,closest_entrance,distance_north,distance_north,distance_north_east,distance_east,distance_west,distance_north_west
12,inhorgenta,2025-02-23,laufzeit,A1,1795,955.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north,600.0,600.0,730.0,760.0,1060.0,1000.0
41,Münchner Autotage,2025-02-23,laufzeit,B3,1000,2000.0,3000.0,P9 - P12,600.0,north,600.0,600.0,670.0,670.0,1190.0,1230.0
67,fre.e + IMOT,2025-02-23,laufzeit,B4,12724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,Lopec,2025-02-23,aufbau,C6,140,360.0,500.0,P2 Nord (östl. Tor 11c),440.0,north_east,520.0,520.0,440.0,560.0,1460.0,1400.0


In [12]:
# Verify, that there are no negative parking delta
if (df_events_allocated_parking_lot['parking_delta'] < 0).any():
    print(df_events_allocated_parking_lot[df_events_allocated_parking_lot['parking_delta'] < 0])
else:
    print('No event demand exceeds allocated parking capacity')

No event demand exceeds allocated parking capacity
